[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Etiquetado_openai.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [1]:
!pip install -q PyPDF2 python-docx openpyxl python-pptx xlrd transformers office365-rest-python-client msal requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 20.5 MB/s eta 0:00:00


In [2]:
import os
import io
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader
import docx
import openpyxl
from pptx import Presentation
import xlrd
from transformers import pipeline
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import os
import requests
import msal
import csv
import getpass
from openai import OpenAI
from transformers import pipeline
import time
import openai
import requests
import pandas as pd

In [3]:
api_key = getpass.getpass("Introduce tu OpenAI API Key: ")
client = OpenAI(api_key=api_key)
token = getpass.getpass("Introduce tu GitHub token: ")

Introduce tu OpenAI API Key: ··········
Introduce tu GitHub token: ··········


In [4]:
!apt-get install git -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


# Configuración

In [5]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"  # tu nuevo Client ID
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]

ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
classifier = pipeline("zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli")
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
!git config --global user.email "marcomendieta08@gmail.com"
!git config --global user.name "marcomendieta08"
!git clone https://github.com/pugapatricia/gestion-documentaria-para-pymes.git

Cloning into 'gestion-documentaria-para-pymes'...
remote: Enumerating objects: 342, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 342 (delta 78), reused 25 (delta 12), pack-reused 208 (from 1)
Receiving objects: 100% (342/342), 6.52 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (184/184), done.


# Conección con OneDrive


In [7]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()

To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code AF4NM74E to authenticate.


#Funciones

In [8]:
ticketsTXT = "https://raw.githubusercontent.com/pugapatricia/gestion-documentaria-para-pymes/refs/heads/main/etiquetado/tickers.txt"
response = requests.get(ticketsTXT)
etiquetas = response.text.strip().split(", ")
print(etiquetas)

['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad', 'arqueología', 'financiero', 'contaduría', 'restitución', 'materias', 'regresión', 'extranjero', 'renta', 'análisis', 'puente', 'servicios', 'inquilino', 'optativa', 'arrendador', 'seguridad', 'notificaciones', 'vigilancia', 'depreciación', 'empresa', 'geografía', 'monográficos', 'estancia', 'balance', 'divulgación', 'autorización', 'master', 'madrid', 'interpretación', 'ingresos', 'métodos', 'fuentes', 'idiomas', 'silvopastoril', 'extranjería', 'receptor', 'proyecto', 'interés', 'apalancamiento', 'propiedad', 'incumplimiento', 'estudios', 'nie', 'datascience', 'complutense', 'mediación', 'firma', 'liquidez', 'valoracion', 'desalojo', 'compraventa', 'alquiler', 'sociales', 'jarama', 'partes', 'contrato', 'tasas', 'humanidades', 'estudiantes', 'activos', 'actividad', 'pasivos', 'financieros', 'registro', 'divulgador', 'cesión', 'poesía', 'temporada', 'documentación', 'rentabilidad', 'comisiones', 'flujodecaja', 'patri

Lector de documentos

In [9]:
def leer_pdf(contenido, limite_palabras):
    texto = ""
    reader = PdfReader(io.BytesIO(contenido))
    for page in reader.pages:
        if page.extract_text():
            texto += page.extract_text() + "\n"
            if len(texto.split()) >= limite_palabras:
                break
    return texto


def leer_docx(contenido, limite_palabras):
    texto = ""
    doc = docx.Document(io.BytesIO(contenido))
    for p in doc.paragraphs:
        if p.text.strip():
            texto += p.text + "\n"
            if len(texto.split()) >= limite_palabras:
                break
    return texto


def leer_excel(contenido, limite_palabras):
    texto = ""
    wb = openpyxl.load_workbook(io.BytesIO(contenido), data_only=True, read_only=True)
    for sheet in wb.worksheets:
        for row in sheet.iter_rows(values_only=True):
            texto += " ".join([str(cell) for cell in row if cell]) + "\n"
            if len(texto.split()) >= limite_palabras:
                break
        if len(texto.split()) >= limite_palabras:
            break
    return texto


def leer_xls(contenido, limite_palabras):
    texto = ""
    temp_file = "temp.xls"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    wb = xlrd.open_workbook(temp_file)
    for sheet in wb.sheets():
        for row_idx in range(sheet.nrows):
            row = sheet.row_values(row_idx)
            texto += " ".join([str(cell) for cell in row if cell]) + "\n"
            if len(texto.split()) >= limite_palabras:
                break
        if len(texto.split()) >= limite_palabras:
            break
    os.remove(temp_file)
    return texto


def leer_pptx(contenido, limite_palabras):
    texto = ""
    temp_file = "temp.pptx"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    prs = Presentation(temp_file)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                texto += shape.text + "\n"
                if len(texto.split()) >= limite_palabras:
                    break
        if len(texto.split()) >= limite_palabras:
            break
    os.remove(temp_file)
    return texto


def leer_txt_csv(contenido, limite_palabras):
    texto = contenido.decode("utf-8", errors="ignore")
    return " ".join(texto.split()[:limite_palabras])

def leer_archivo(nombre, contenido, limite_palabras=None):
    ext = nombre.split(".")[-1].lower()
    if ext == "pdf":
        return leer_pdf(contenido, limite_palabras)
    elif ext == "docx":
        return leer_docx(contenido, limite_palabras)
    elif ext == "xlsx":
        return leer_excel(contenido, limite_palabras)
    elif ext == "xls":
        return leer_xls(contenido, limite_palabras)
    elif ext == "pptx":
        return leer_pptx(contenido, limite_palabras)
    elif ext in {"txt", "csv"}:
        return leer_txt_csv(contenido, limite_palabras)
    return ""

Clasificador Hugging Face Zero-Shot


In [10]:
def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = texto.lower()
    texto = re.sub(r"[\r\n\t]+", " ", texto)
    texto = re.sub(r"[^a-záéíóúüñ0-9\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

def etiquetar_texto(texto, umbral=0.3, min_etiquetas=5, max_etiquetas=10):
    res = classifier(texto, candidate_labels=etiquetas, multi_label=True)
    etiquetas_filtradas = [label for label, score in zip(res["labels"], res["scores"]) if score >= umbral]
    if len(etiquetas_filtradas) < min_etiquetas:

        etiquetas_ordenadas = [label for _, label in sorted(zip(res["scores"], res["labels"]), reverse=True)]
        for label in etiquetas_ordenadas:
            if label not in etiquetas_filtradas:
                etiquetas_filtradas.append(label)
            if len(etiquetas_filtradas) >= min_etiquetas:
                break
    etiquetas_filtradas = etiquetas_filtradas[:max_etiquetas]

    return etiquetas_filtradas


In [11]:
def generar_descripcion(texto, max_palabras=300):
    try:
        # Tomar solo las primeras N palabras para no sobrecargar al modelo
        texto_corto = " ".join(texto.split()[:max_palabras])
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Eres un asistente que resume documentos brevemente en una sola línea."},
                {"role": "user", "content": f"Resume el siguiente texto en una sola línea:\n\n{texto_corto}"}
            ],
            max_completion_tokens=50  # suficiente para una línea
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generando descripción: {e}"


# Leer archivos de OneDrive

In [33]:
resultados = {}
descripciones = {}

for item in data.get("value", []):
    nombre = item["name"]
    if not any(nombre.lower().endswith(ext) for ext in ext_permitidas):
        continue
    download_url = item["@microsoft.graph.downloadUrl"]
    file_bytes = requests.get(download_url).content

    # Leer archivo según extensión
    texto = leer_archivo(nombre, file_bytes, limite_palabras = 300)
    texto = limpiar_texto(texto)

    if texto:
        resultados[nombre] = etiquetar_texto(texto)
        descripciones[nombre] = generar_descripcion(texto)
    else:
        resultados[nombre] = []
        descripciones[nombre] = "No se pudo leer el archivo"

# Crear DataFrame final
df_final = pd.DataFrame({
    "Archivo": list(resultados.keys()),
    "Etiquetas": list(resultados.values()),
    "Descripcion": [descripciones[a] for a in resultados.keys()]
})


In [28]:
df_final.to_csv("etiquetas_onedrive.csv", index=False, encoding="utf-8-sig")

from google.colab import files
files.download("etiquetas_onedrive.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Guardar resultados

In [29]:
%cd /content/gestion-documentaria-para-pymes

/content/gestion-documentaria-para-pymes


In [30]:

os.makedirs("etiquetado", exist_ok=True)
csv_path = "etiquetado/etiquetas_onedrive.csv"
with open(csv_path, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Archivo", "Etiquetas"])
    for nombre, etiquetas_detectadas in resultados.items():
        etiquetas_str = ", ".join(etiquetas_detectadas) if etiquetas_detectadas else "Sin etiquetas"
        writer.writerow([nombre, etiquetas_str])
print(f"✅ Resultados guardados en {csv_path}")


✅ Resultados guardados en etiquetado/etiquetas_onedrive.csv


In [31]:
!ls etiquetado/

Etiquetado_huggingface.ipynb  Generador_etiquetasHF.ipynb
Etiquetado_openai.ipynb       Generador_etiquetas.ipynb
etiquetas_onedrive.csv	      tickers.txt
flujo-de-proceso.md	      UnionEtiquetado_open.ipynb


In [32]:
!git add etiquetado/etiquetas_onedrive.csv
!git diff --cached --quiet || git commit -m "Actualizar etiquetas_onedrive.csv desde Colab"
!git pull https://{token}@github.com/pugapatricia/gestion-documentaria-para-pymes.git main
!git push https://{token}@github.com/pugapatricia/gestion-documentaria-para-pymes.git main

From https://github.com/pugapatricia/gestion-documentaria-para-pymes
 * branch            main       -> FETCH_HEAD
Already up to date.
Everything up-to-date


## Codigo para cargar en la etiqueta de descripcion en onedrive

In [18]:
"""
# ============================
# Configuración
# ============================
json_path = "etiquetas_onedrive.json"
headers = {"Authorization": f"Bearer {access_token}"}

# Carpeta objetivo en OneDrive
carpeta_objetivo = "Etiquetados"

# ============================
# Cargar JSON de etiquetas
# ============================
with open(json_path, "r", encoding="utf-8") as f:
    etiquetas_data = json.load(f)

# ============================
# Aplicar etiquetas en la descripción
# ============================
for archivo, etiquetas in etiquetas_data.items():
    if not etiquetas:
        continue

    # Buscar archivo en OneDrive
    url_file = f"https://graph.microsoft.com/v1.0/me/drive/root:/{carpeta_objetivo}/{archivo}"
    resp_file = requests.get(url_file, headers=headers)

    if resp_file.status_code != 200:
        print(f"⚠️ No se encontró {archivo} en OneDrive")
        continue

    file_id = resp_file.json()["id"]

    # Guardamos etiquetas en el campo "description"
    url_update = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}"
    payload = {"description": ", ".join(etiquetas)}

    response_update = requests.patch(
        url_update,
        headers={**headers, "Content-Type": "application/json"},
        json=payload
    )

    if response_update.status_code in [200, 204]:
        print(f"✅ Etiquetas {etiquetas} aplicadas a {archivo} en la descripción")
    else:
        print(f"⚠️ Error al actualizar {archivo}: {response_update.text}")
"""

'\n# ============================\n# Configuración\n# ============================\njson_path = "etiquetas_onedrive.json"\nheaders = {"Authorization": f"Bearer {access_token}"}\n\n# Carpeta objetivo en OneDrive\ncarpeta_objetivo = "Etiquetados"\n\n# ============================\n# Cargar JSON de etiquetas\n# ============================\nwith open(json_path, "r", encoding="utf-8") as f:\n    etiquetas_data = json.load(f)\n\n# ============================\n# Aplicar etiquetas en la descripción\n# ============================\nfor archivo, etiquetas in etiquetas_data.items():\n    if not etiquetas:\n        continue\n\n    # Buscar archivo en OneDrive\n    url_file = f"https://graph.microsoft.com/v1.0/me/drive/root:/{carpeta_objetivo}/{archivo}"\n    resp_file = requests.get(url_file, headers=headers)\n\n    if resp_file.status_code != 200:\n        print(f"⚠️ No se encontró {archivo} en OneDrive")\n        continue\n\n    file_id = resp_file.json()["id"]\n\n    # Guardamos etiquetas en 